### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

In [1]:
import numpy as np

#### generate a simple linear dataset y=xb+e
##### store validation data from same dataset too

In [2]:
import data_gen as dg
gen_params=2
data1=dg.data_gen(data_shape=(500,gen_params),seed=1,ftype='quadratic',evar=3)
#xdata=data1.x[:,:-1]#drop last x
size=30
xdata=data1.x[0:size,:gen_params+1]
ydata=data1.y[0:size]
y=ydata
xfull=xdata
x=xdata[:,1:] #drop first column of x (all 1's)
e=data1.e
n,p=x.shape


xvalidate=data1.x[100:500]
yvalidate=data1.y[100:500]

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
  - 'product'-Ndiff1 multiplied by Ndiff2
  - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
  - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
  - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- Ndiff_bw_kern:
  - rbfkern means use the radial basis function kernel
  - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
  - 'NW' means use nadaraya-watson kernel regression
  - 'full_logit' means local logit with all variables entering linearly
  - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- ykern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data points (self is masked when predicting self)
- xkern_grid
  - much like ykern_grid, but used less frequently since x's are typically pre-specified
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels across the nout axis (the number of possibile values y is averaged over, whose probabilities sum to 1)
  - 'own_n' means same as self, but divided by count of non-masked items in second to last, nout lenght dimension
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - x_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [3]:
max_bw_Ndiff=2
Ndiff_start=1
Ndiff_param_count=max_bw_Ndiff-(Ndiff_start-1)
modeldict1={
    'Ndiff_type':'product',
    'Ndiff_start':Ndiff_start,
    'max_bw_Ndiff':max_bw_Ndiff,
    'normalize_Ndiffwtsum':'own_n',
    'xkern_grid':'no',
    'ykern_grid':'no',
    'outer_kern':'gaussian',
    'Ndiff_bw_kern':'rbfkern',
    'outer_x_bw_form':'one_for_all',
    'regression_model':'NW',
    'product_kern_norm':'self',
    'hyper_param_form_dict':{
        'Ndiff_exponent':'free',
        'x_bandscale':'non-neg',
        'Ndiff_depth_bw':'non-neg',
        'outer_x_bw':'non-neg',
        'outer_y_bw':'non-neg',
        'y_bandscale':'non-neg'
        }
    }

modeldict1['ykern_grid']=35#overridem
modeldict1['Ndiff_type']='recursive'
#modeldict1['product_kern_norm']='none'

#set starting or fixed values for hyper parameters

#if max_bw_Ndiff>0:onethenzeros[0]=1
hyper_paramdict1={
    'Ndiff_exponent':np.array([0.3,-0.03]),
    'x_bandscale':-0.3*np.ones([p,]),
    'outer_x_bw':np.array([2.7,]),
    'outer_y_bw':np.array([2.2,]),
    'Ndiff_depth_bw':np.array([0.4,1.07]),
    'y_bandscale':1.5*np.ones([1,])
    }

#try recursive:
if modeldict1['Ndiff_type']=='recursive':
    hyper_paramdict1={
        'Ndiff_exponent':np.array([0.3,0.3]),
        'x_bandscale':-0.3*np.ones([p,]),
        'outer_x_bw':np.array([2.7,]),
        'outer_y_bw':np.array([2.2,]),
        'Ndiff_depth_bw':np.array([0.4]),
        'y_bandscale':1.5*np.ones([1,])
        }



#assert len(hyper_paramdict1['Ndiff_exponent'])==modeldict1['max_bw_Ndiff'],'Ndiff_exponent does not match ' \
#                                                                            'depth of max_bw_Ndiff'
#assert len(hyper_paramdict1['Ndiff_depth_bw'])==modeldict1['max_bw_Ndiff'],'Ndiff_depth_bw does not match ' \
#                                                                             'depth of max_bw_Ndiff'
#create optimization dictionary
optimizedict1={
    'optmz_dict':
    'hyper_param_dict':hyper_paramdict1,
    'model_dict':modeldict1
    }

In [4]:
print(Ndiff_param_count)

2


#-----------------Calibrate/Optimize--------

In [5]:
import mykern as mk



In [6]:
import myreg
just_linreg=myreg.reg(y,xfull)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)

linreg_mse: 1148.6713696695078


In [7]:
##### y=np.linspace(0,10000,n)#what happens if y has no relationship to x
optimized_Ndiff_kernel=mk.optimize_free_params(y,x,optimizedict1)
print(optimized_Ndiff_kernel.mselist[-1])

iter:2,mse:(404.4009334876519, {'Ndiff_exponent': {'value': array([0.3, 0.3]), 'const': 'free'}, 'x_bandscale': {'value': array([-0.3, -0.3]), 'const': 'non-neg:[0.74081822 0.74081822]'}, 'Ndiff_depth_bw': {'value': array([0.4]), 'const': 'non-neg:[1.4918247]'}, 'outer_x_bw': {'value': array([2.7]), 'const': 'non-neg:[14.87973172]'}, 'outer_y_bw': {'value': array([2.2]), 'const': 'non-neg:[9.0250135]'}, 'y_bandscale': {'value': array([1.5]), 'const': 'non-neg:[4.48168907]'}})
iter:3,mse:(371.0381108317331, {'Ndiff_exponent': {'value': array([0.315, 0.3  ]), 'const': 'free'}, 'x_bandscale': {'value': array([-0.3, -0.3]), 'const': 'non-neg:[0.74081822 0.74081822]'}, 'Ndiff_depth_bw': {'value': array([0.4]), 'const': 'non-neg:[1.4918247]'}, 'outer_x_bw': {'value': array([2.7]), 'const': 'non-neg:[14.87973172]'}, 'outer_y_bw': {'value': array([2.2]), 'const': 'non-neg:[9.0250135]'}, 'y_bandscale': {'value': array([1.5]), 'const': 'non-neg:[4.48168907]'}})
iter:4,mse:(405.34835034681885, {'

In [8]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [9]:
#xgrid=np.linspace(1,n,n)

xgrid=x[:,0]
yhat=optimized_Ndiff_kernel.yhat_un_std
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=10,color='blue',alpha=0.4,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',alpha=0.9,legend='yhat')
p.circle(xgrid,just_linreg.yhat,color='red',legend='just lin reg yhat')
p.line(xgrid,np.mean(y))
p.legend.location = "top_left"
#p.yaxis.visible=False
show(p)

In [10]:
yhat.shape
print(data1.b)

[3 8 9 8 5]


In [11]:
print(yhat)

[71.89832608821543 134.92187024046473 68.32869781588364 15.343704906620594
 135.4480594897828 36.693378330082396 53.042527303228724 45.77968030546976
 8.242380034113872 93.19779333730321 18.30516047668403 30.612259636265595
 46.340313925566846 17.659311276444296 56.5819838573566 70.16605944512645
 44.956834531793675 87.86517615491206 51.84569769842433 44.18174512954397
 97.83563411126667 24.875721010137777 76.09053372508703 8.313138490349928
 10.599961918251424 75.79636820849282 12.620680426148255 6.533157757421478
 7.650053408712907 74.06151300954778]


In [12]:
print(y)

[ 69.3591003  136.02155891  66.44174278   2.52470806 136.47820781
  29.00683338  49.96993552  41.78974389   6.37811171  90.86345352
   0.4900605   23.40481953  42.62185345  16.46834833  53.06895582
  65.6734997   39.43637174  78.7047258   43.28390874  39.57256655
  85.10294898  20.11551243  66.00201883   6.05187519   6.79911392
  95.2306912   13.57522898   7.68340314   5.94050922  64.84707916]


In [13]:

ypredict=optimized_Ndiff_kernel.predict_tool(xvalidate,optimized_Ndiff_kernel.fixed_or_free_paramdict,modeldict1)
prediction_err=yvalidate-ypredict
prediction_mse=np.mean(prediction_err**2)

ValueError: operands could not be broadcast together with shapes (400,5) (2,) 

In [ ]:
np.ma.count_masked(yhat)

In [ ]:
class toolcompare(object):